# Creating TCGA cohorts  (part 1)

This notebook will show you how to create a TCGA cohort using the publicly available TCGA BigQuery tables that the [ISB-CGC](http://isb-cgc.org) project has produced based on the open-access [TCGA](http://cancergenome.nih.gov/) data available at the [Data Portal](https://tcga-data.nci.nih.gov/tcga/).  You will need to have access to a Google Cloud Platform (GCP) project in order to use BigQuery.  If you don't already have one, you can sign up for a [free-trial](https://cloud.google.com/free-trial/) or contact [us](mailto://info@isb-cgc.org) and become part of the community evaluation phase of our [Cancer Genomics Cloud pilot](https://cbiit.nci.nih.gov/ncip/nci-cancer-genomics-cloud-pilots).

We are not attempting to provide a thorough BigQuery or IPython tutorial here, as a wealth of such information already exists.  Here are some links to some resources that you might find useful: 
* [BigQuery](https://cloud.google.com/bigquery/what-is-bigquery), 
* the BigQuery [web UI](https://bigquery.cloud.google.com/) where you can run queries interactively, 
* [IPython](http://ipython.org/) (now known as [Jupyter](http://jupyter.org/)), and 
* [Cloud Datalab](https://cloud.google.com/datalab/) the recently announced interactive cloud-based platform that this notebook is being developed on. 

There are also many tutorials and samples available on github (see, in particular, the [datalab](https://github.com/GoogleCloudPlatform/datalab) repo and the [Google Genomics](  https://github.com/googlegenomics) project).

OK then, let's get started!  In order to work with BigQuery, the first thing you need to do is import the bigquery module:

In [1]:
import google.cloud.bigquery as bq
client = bq.Client()

The next thing you need to know is how to access the specific tables you are interested in.  BigQuery tables are organized into datasets, and datasets are owned by a specific GCP project.  The tables we will be working with in this notebook are in a dataset called **`tcga_201607_beta`**, owned by the **`isb-cgc`** project.  A full table identifier is of the form `<project_id>:<dataset_id>.<table_id>`.  Let's start by getting some basic information about the tables in this dataset:

In [6]:
client = bq.Client(project='isb-cgc')
dataset = bq.Dataset('isb-cgc.tcga_201607_beta')
dataset_ref = client.dataset('tcga_201607_beta')
tablestr = "isb-cgc.tcga_201607_beta.Clinical_data"

tables = list(client.list_tables(dataset))  # API request(s)
for table in tables:
   table_ref = dataset_ref.table(table.table_id)
   table = client.get_table(table_ref)  # API Request
   print('Table ID {0:30s} {1:10d}'.format(table.table_id, table.num_rows))

Table ID Annotations                          6322
Table ID Biospecimen_data                    23797
Table ID Clinical_data                       11160
Table ID Copy_Number_segments              2646095
Table ID DNA_Methylation_betas          3944304319
Table ID DNA_Methylation_chr1            382335670
Table ID DNA_Methylation_chr10           197519895
Table ID DNA_Methylation_chr11           235823572
Table ID DNA_Methylation_chr12           198050739
Table ID DNA_Methylation_chr13            97301675
Table ID DNA_Methylation_chr14           123239379
Table ID DNA_Methylation_chr15           124566185
Table ID DNA_Methylation_chr16           179772812
Table ID DNA_Methylation_chr17           234003341
Table ID DNA_Methylation_chr18            50216619
Table ID DNA_Methylation_chr19           211386795
Table ID DNA_Methylation_chr2            279668485
Table ID DNA_Methylation_chr20            86858120
Table ID DNA_Methylation_chr21            35410447
Table ID DNA_Methylation_chr22 

In this tutorial, we are going to look at a few different ways that we can use the information in these tables to create cohorts.  Now, you maybe asking what we mean by "cohort" and why you might be interested in *creating* one, or maybe what it even means to "create" a cohort.  The TCGA dataset includes clinical, biospecimen, and molecular data from over 10,000 cancer patients who agreed to be a part of this landmark research project to build [The Cancer Genome Atlas](http://cancergenome.nih.gov/).  This large dataset was originally organized and studied according to [cancer type](http://cancergenome.nih.gov/cancersselected) but now that this multi-year project is nearing completion, with over 30 types of cancer and over 10,000 tumors analyzed, **you** have the opportunity to look at this dataset from whichever angle most interests you.  Maybe you are particularly interested in early-onset cancers, or gastro-intestinal cancers, or a specific type of genetic mutation.  This is where the idea of a "cohort" comes in.  The original TCGA "cohorts" were based on cancer type (aka "study"), but now you can define a cohort based on virtually any clinical or molecular feature by querying these BigQuery tables.  A cohort is simply a list of samples, using the [TCGA barcode](https://wiki.nci.nih.gov/display/TCGA/TCGA+barcode) system.  Once you have created a cohort you can use it in any number of ways: you could further explore the data available for one cohort, or compare one cohort to another, for example.

In the rest of this tutorial, we will create several different cohorts based on different motivating research questions.  We hope that these examples will provide you with a starting point from which you can build, to answer your own research questions.

### Exploring the Clinical data table
Let's start by looking at the clinical data table.  The TCGA dataset contains a few very basic clinical data elements for almost all patients, and contains additional information for some tumor types only.  For example smoking history information is generally available only for lung cancer patients, and BMI (body mass index) is only available for tumor types where that is a known significant risk factor.  Let's take a look at the clinical data table and see how many different pieces of information are available to us:

In [86]:
%%bash
bq show --schema isb-cgc:tcga_201607_beta.Clinical_data

[{"type":"STRING","name":"ParticipantBarcode"},{"type":"STRING","name":"Study"},{"type":"STRING","name":"Project"},{"type":"STRING","name":"ParticipantUUID"},{"type":"STRING","name":"TSSCode"},{"type":"INTEGER","name":"age_at_initial_pathologic_diagnosis"},{"type":"STRING","name":"anatomic_neoplasm_subdivision"},{"type":"INTEGER","name":"batch_number"},{"type":"STRING","name":"bcr"},{"type":"STRING","name":"clinical_M"},{"type":"STRING","name":"clinical_N"},{"type":"STRING","name":"clinical_T"},{"type":"STRING","name":"clinical_stage"},{"type":"STRING","name":"colorectal_cancer"},{"type":"STRING","name":"country"},{"type":"STRING","name":"vital_status"},{"type":"INTEGER","name":"days_to_birth"},{"type":"INTEGER","name":"days_to_death"},{"type":"INTEGER","name":"days_to_last_known_alive"},{"type":"INTEGER","name":"days_to_last_followup"},{"type":"INTEGER","name":"days_to_initial_pathologic_diagnosis"},{"type":"INTEGER","name":"days_to_submitted_specimen_dx"},{"type":"STRING","name":"eth

That's a lot of fields!  We can also get at the schema programmatically:

In [4]:
table_ref = dataset_ref.table('Clinical_data')
table = client.get_table(table_ref)  # API Request
#table = bq.Table('isb-cgc:tcga_201607_beta.Clinical_data')
#table = client.get_table(table_ref)
#if ( table.exists() ):
#print("Schema is {}".format(table.schema))
fieldNames = map(lambda tsf: tsf.name, table.schema)
fieldTypes = map(lambda tsf: tsf.field_type, table.schema)
listfieldNames = list(fieldNames)
listfieldTypes = list(fieldTypes)

print("This table has {} fields. ".format( len(listfieldNames) ))
print("The first few field names and types are: ") 
print(listfieldNames[:5])
print(listfieldTypes[:5])

This table has 70 fields. 
The first few field names and types are: 
['ParticipantBarcode', 'Study', 'Project', 'ParticipantUUID', 'TSSCode']
['STRING', 'STRING', 'STRING', 'STRING', 'STRING']


Let's look at these fields and see which ones might be the most "interesting", by looking at how many times they are filled-in (not NULL), or how much variation exists in the values.  If we wanted to look at just a single field, "tobacco_smoking_history" for example, we could use a very simple query to get a basic summary:

In [88]:
%%bigquery 

SELECT tobacco_smoking_history, COUNT(*) AS n
FROM `isb-cgc.tcga_201607_beta.Clinical_data`
GROUP BY tobacco_smoking_history
ORDER BY n DESC

,tobacco_smoking_history,n
0,None,8161
1,1,865
2,4,799
3,2,710
4,3,568
5,5,57


But if we want to loop over *all* fields and get a sense of which fields might provide us with useful criteria for specifying a cohort, we'll want to automate that.  We'll put a threshold on the minimum number of patients that we expect information for, and the maximum number of unique values (since fields such as the "ParticipantBarcode" will be unique for every patient and, although we will need that field later, it's probably not useful for defining a cohort).

In [ ]:
client = bq.Client()
numPatients = table.num_rows
print(" The {} table describes a total of {} patients. ".format(table.table_id, table.num_rows) )

# let's set a threshold for the minimum number of values that a field should have,
# and also the maximum number of unique values
minNumPatients = int(numPatients*0.80)
maxNumValues = 50


numInteresting = 0
iList = []
print("Field Names list is {}".format(len(listfieldNames)))
for iField in range(len(listfieldNames)):
  aField = listfieldNames[iField]
  aType = listfieldTypes[iField]
  try:
    qString = "SELECT {0} FROM `{1}`".format(aField,tablestr)
    df = client.query(qString).to_dataframe()
    summary = df[str(aField)].describe()
    if ( aType == "STRING" ):
      topFrac = float(summary['freq'])/float(summary['count'])
      if ( summary['count'] >= minNumPatients ):
        if ( summary['unique'] <= maxNumValues and summary['unique'] > 1 ):
          if ( topFrac < 0.90 ):
            numInteresting += 1
            iList += [aField]
            print("   > {} has {} values with {} unique ({} occurs {} times) ". \
              format(str(aField), summary['count'], summary['unique'], summary['top'], summary['freq']))
    else:
      if ( summary['count'] >= minNumPatients ):
        if ( summary['std'] > 0.1 ):
          numInteresting += 1
          iList += [aField]
          print("     > {} has {} values (mean={:0}, sigma={:0} ". \
            format(str(aField), summary['count'], summary['mean'], summary['std']))
  except:
    pass    

print(" ")
print(" Found {} potentially interesting features: ".format(numInteresting))
print("   ", iList)

 The miRNA_Expression table describes a total of 13226928 patients. 
Field Names list is 70
SELECT ParticipantBarcode FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT Study FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT Project FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT ParticipantUUID FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT TSSCode FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT age_at_initial_pathologic_diagnosis FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT anatomic_neoplasm_subdivision FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT batch_number FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT bcr FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT clinical_M FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT clinical_N FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT clinical_T FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT clinical_stage FROM `isb-cgc.tcga_201607_beta.Clinical_data`
SELECT colorectal_cancer FROM `

In [ ]:
query = """SELECT ParticipantBarcode FROM `isb-cgc.tcga_201607_beta.Clinical_data` LIMIT 5"""
#results = client.query(query).result()

df = client.query(query).to_dataframe()
summary = df[str("ParticipantBarcode")].describe()
print(summary)
#for row in results:
#    print("{} : ".format(row.ParticipantBarcode))



The above helps us narrow down on which fields are likely to be the most useful, but if you have a specific interest, for example in menopause or HPV status, you can still look at those in more detail very easily: 

In [ ]:
%%bigquery
SELECT menopause_status, COUNT(*) AS n
FROM `isb-cgc.tcga_201607_beta.Clinical_data`
WHERE menopause_status IS NOT NULL
GROUP BY menopause_status
ORDER BY n DESC

We might wonder which specific tumor types have menopause information:

In [ ]:
%%bigquery
SELECT Study, COUNT(*) AS n
FROM `isb-cgc.tcga_201607_beta.Clinical_data`
WHERE menopause_status IS NOT NULL
GROUP BY Study
ORDER BY n DESC

In [ ]:
%%bigquery
SELECT hpv_status, hpv_calls, COUNT(*) AS n
FROM `isb-cgc.tcga_201607_beta.Clinical_data`
WHERE hpv_status IS NOT NULL
GROUP BY hpv_status, hpv_calls
HAVING n > 20
ORDER BY n DESC

### TCGA Annotations

An additional factor to consider, when creating a cohort is that there may be additional information that might lead one to exclude a particular patient from a cohort.  In certain instances, patients have been redacted or excluded from analyses for reasons such as prior treatment, etc, but since different researchers may have different criteria for using or excluding certain patients or certain samples from their analyses, in many cases the data is still available while at the same time "annotations" may have been entered into a searchable [database](https://tcga-data.nci.nih.gov/annotations/).  These annotations have also been uploaded into a BigQuery table and can be used in conjuction with the other BigQuery tables.

### Early-onset Breast Cancer

Now that we have a better idea of what types of information is available in the Clinical data table, let's create a cohort consisting of female breast-cancer patients, diagnosed at the age of 50 or younger.

In this next code cell, we define several queries within a **`module`** which allows us to use them both individually and by reference in the final, main query.  
+ the first query, called **`select_on_annotations`**, finds all patients in the Annotations table which have either been 'redacted' or had 'unacceptable prior treatment';  
+ the second query, **`select_on_clinical`** selects all female breast-cancer patients who were diagnosed at age 50 or younger, while also pulling out a few additional fields that might be of interest;  and
+ the final query joins these two together and returns just those patients that meet the clinical-criteria and do **not** meet the exclusion-criteria.

In [113]:
%%bigquery 
WITH 
  select_on_annotations AS (
    SELECT
      ParticipantBarcode,
      annotationCategoryName AS categoryName,
      annotationClassification AS classificationName
    FROM
      `isb-cgc.tcga_201607_beta.Annotations`
    WHERE
      ( itemTypeName="Patient"
        AND (annotationCategoryName="History of unacceptable prior treatment related to a prior/other malignancy"
          OR annotationClassification="Redaction" ) )
    GROUP BY
      ParticipantBarcode,
      categoryName,
      classificationName
  ),
  select_on_clinical AS (
    SELECT
      ParticipantBarcode,
      vital_status,
      days_to_last_known_alive,
      ethnicity,
      histological_type,
      menopause_status,
      race
    FROM
      `isb-cgc.tcga_201607_beta.Clinical_data`
    WHERE
      ( Study="BRCA"
        AND age_at_initial_pathologic_diagnosis<=50
        AND gender="FEMALE" )
  )
SELECT
  ac.ParticipantBarcode AS ParticipantBarcode
FROM  (
  SELECT
    a.categoryName,
    a.classificationName,
    c.ParticipantBarcode
  FROM select_on_annotations AS a
  FULL OUTER JOIN select_on_clinical AS c
  ON
    a.ParticipantBarcode = c.ParticipantBarcode
  WHERE
    (a.ParticipantBarcode IS NOT NULL
      OR c.ParticipantBarcode IS NOT NULL )
  ORDER BY
    a.classificationName,
    a.categoryName,
    a.ParticipantBarcode,
    c.ParticipantBarcode ) AS ac
WHERE
  ( ac.categoryName IS NULL
    AND ac.classificationName IS NULL
    AND ac.ParticipantBarcode IS NOT NULL )
ORDER BY
  ac.ParticipantBarcode

,ParticipantBarcode
0,TCGA-3C-AALI
1,TCGA-4H-AAAK
2,TCGA-5L-AAT0
3,TCGA-A1-A0SH
4,TCGA-A1-A0SJ
5,TCGA-A1-A0SN
6,TCGA-A1-A0SP
7,TCGA-A1-A0SQ
8,TCGA-A2-A04P
9,TCGA-A2-A04Q


Here we explicitly call just the first query in the module, and we get a list of 212 patients with one of these disqualifying annotations:

In [118]:
#bq.Query(createCohort_and_checkAnnotations.select_on_annotations).results().to_dataframe()

select_on_annotations = """
    SELECT
      ParticipantBarcode,
      annotationCategoryName AS categoryName,
      annotationClassification AS classificationName
    FROM
      `isb-cgc.tcga_201607_beta.Annotations`
    WHERE
      ( itemTypeName="Patient"
        AND (annotationCategoryName="History of unacceptable prior treatment related to a prior/other malignancy"
          OR annotationClassification="Redaction" ) )
    GROUP BY
      ParticipantBarcode,
      categoryName,
      classificationName
"""
df = client.query(select_on_annotations).to_dataframe()

and here we explicitly call just the second query, resulting in 329 patients:

In [119]:
#bq.Query(createCohort_and_checkAnnotations.select_on_clinical).results().to_dataframe()
select_on_clinical = """
    SELECT
      ParticipantBarcode,
      vital_status,
      days_to_last_known_alive,
      ethnicity,
      histological_type,
      menopause_status,
      race
    FROM
      `isb-cgc.tcga_201607_beta.Clinical_data`
    WHERE
      ( Study="BRCA"
        AND age_at_initial_pathologic_diagnosis<=50
        AND gender="FEMALE" )
"""
df = client.query(select_on_clinical).to_dataframe()

and finally we call the main query:

In [ ]:
#TBD
df = bq.Query(createCohort_and_checkAnnotations).results().to_dataframe()

Note that we didn't need to call each sub-query individually, we could have just called the main query and gotten the same result.  As you can see, two patients that met the clinical select criteria (which returned 329 patients) were excluded from the final result (which returned 327 patients).

Before we leave off, here are a few useful tricks for working with BigQuery in Cloud Datalab:
+ if you want to see the raw SQL, you can just build the query and then print it out (this might be useful, for example, in debugging a query -- you can copy and paste the SQL directly into the BigQuery Web UI);
+ if you want to see how much data and which tables are going to be touched by this data, you can use the "dry run" option.  (Notice the "cacheHit" flag -- if you have recently done a particular query, you will not be charged to repeat it since it will have been cached.)

In [ ]:
q.execute_dry_run()